In [1]:
#This file contains the python implementation feature based change detector
#Author: Bhavan Vasu

import tensorflow as tf
import keras
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
import numpy as np
import matplotlib.pyplot as plt
import sys
from skimage import filters #change to 'import filter' for Python>v2.7
from skimage import exposure
from keras import backend as K
sess = tf.InteractiveSession()

Using TensorFlow backend.


In [2]:
size=112
#Function to retrieve features from intermediate layers
def get_activations(model, layer_idx, X_batch):
    get_activations = K.function([model.layers[0].input, K.learning_phase()], [model.layers[layer_idx].output,])
    activations = get_activations([X_batch,0])
    return activations

#Function to extract features from intermediate layers
def extra_feat(img_path):
        #Using a VGG19 as feature extractor
        base_model = VGG19(weights='imagenet',include_top=False)
        #img = image.load_img(img_path, target_size=(224, 224))
        img = image.load_img(img_path)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        block1_pool_features=get_activations(base_model, 3, x)
        block2_pool_features=get_activations(base_model, 6, x)
        block3_pool_features=get_activations(base_model, 10, x)
        block4_pool_features=get_activations(base_model, 14, x)
        block5_pool_features=get_activations(base_model, 18, x)

        x1 = tf.image.resize_images(block1_pool_features[0],[size,size])
        x2 = tf.image.resize_images(block2_pool_features[0],[size,size])
        x3 = tf.image.resize_images(block3_pool_features[0],[size,size])
        x4 = tf.image.resize_images(block4_pool_features[0],[size,size])
        x5 = tf.image.resize_images(block5_pool_features[0],[size,size])
        
        #F = tf.concat([x3,x2,x1,x4,x5],3) #Change to only x1, x1+x2,x1+x2+x3..so on, inorder to visualize features from diffetrrnt blocks
        F = tf.concat([x1],3)
        return F

In [3]:
if (len(sys.argv))>3:
    print("Invalid number of input arguments ")
exit(0)
#Two aerial patches with change or No change
img_path1='./siamesedata/3.jpg'#sys.argv[1]
img_path2='siamesedata/2.jpg'#sys.argv[2]
i=0


In [4]:
F1=extra_feat(img_path1) #Features from image patch 1
#print(F1.shape)
F1=tf.square(F1)
F2=extra_feat(img_path2) #Features from image patch 2
F2=tf.square(F2)
d=tf.subtract(F1,F2)
#print(d.shape)
d=tf.square(d) 
d=tf.reduce_sum(F1,axis=3) 
"""
F1=extra_feat(img_path1) #Features from image patch 1
#print(F1.shape)
F1=tf.square(F1)
F2=extra_feat(img_path2) #Features from image patch 2
F2=tf.square(F2)
d=tf.subtract(F1,F2)
#print(d.shape)
d=tf.square(d) 
d=tf.reduce_sum(d,axis=3)
"""

FileNotFoundError: [Errno 2] No such file or directory: './siamesedata/3.jpg'

In [ ]:
dis=(d.eval())   #The change map formed showing change at each pixels
#plt.imshow(dis)
#plt.show()
print(dis.shape)
dis=np.resize(dis,[size,size])
#plt.imshow(dis)
#plt.show()
# Calculating threshold using Otsu's Segmentation method
val = filters.threshold_otsu(dis[:,:])
hist, bins_center = exposure.histogram(dis[:,:],nbins=256)

plt.title('Unstructured change')
plt.imshow(dis[:,:] < val, cmap='gray', interpolation='bilinear')
plt.axis('off')
plt.tight_layout()
plt.show()
change=dis[:,:]<val

"""
#Uncomment For veiwing a graph for visualizing threshold selection
plt.subplot(144)
plt.title('Otsu Threshold selection')
plt.plot(bins_center, hist, lw=2)
plt.axvline(val, color='k', ls='--')
plt.tight_layout()
plt.show()
"""